In [7]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

import dagshub

def load_code_blocks(DATASET_PATH, CODE_COLUMN):
    df = pd.read_csv(DATASET_PATH, encoding='utf-8', comment='#', sep=',')#, quoting=csv.QUOTE_NONE, error_bad_lines=False)#, sep=','
    print(df.head())
    code_blocks = df[CODE_COLUMN]
    # test_size = 0.1
    # test_rows = round(df.shape[0]*test_size)
    # train_rows = df.shape[0] - test_rows
    # train_code_blocks = df[CODE_COLUMN][0:test_rows]
    # test_code_blocks = df[CODE_COLUMN][train_rows:]
    return df, code_blocks

def tfidf_fit_transform(code_blocks, params, TFIDF_DIR):
    vectorizer = TfidfVectorizer(**params)
    tfidf = vectorizer.fit(code_blocks)
    pickle.dump(tfidf, open(TFIDF_DIR, "wb"))
    print('TF-IDF model has been saved')
    code_blocks_tfidf = tfidf.transform(code_blocks)
    return code_blocks_tfidf

def SVM_evaluate(df, code_blocks, tfidf_params, TFIDF_DIR, SVM_params):
    code_blocks_tfidf = tfidf_fit_transform(code_blocks, tfidf_params, TFIDF_DIR)
    X_train, X_test, y_train, y_test = train_test_split(code_blocks_tfidf, df[TAG_TO_PREDICT], test_size=0.3)
    # grid = {"C": [100]}
    # cv = KFold(n_splits=2, shuffle=True, random_state=241)
    model = SVC(kernel="linear", random_state=241)
    # gs = GridSearchCV(model, grid, scoring="accuracy", cv=cv, verbose=1, n_jobs=-1)
    # gs.fit(X_train[:25000], y_train.ravel()[:25000])
    # C = gs.best_params_.get('C')
    # model = SVC(**SVM_params)
    print("Train SVM params:", model.get_params())
    n_estimators = 10
    clf = BaggingClassifier(model, max_samples=1.0 / n_estimators, n_estimators=n_estimators)
    # clf = model
    print("starting training..")
    clf.fit(X_train, y_train)
    print("saving the model")
    pickle.dump(clf, open(MODEL_DIR, 'wb'))
    print("predicting on the test..")
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    # confus_matrix = confusion_matrix(model, X_test, y_test)
    metrics = {'test_accuracy': accuracy
            , 'test_f1_score': f1}
    print(metrics)
    return metrics

In [8]:
# if __name__ == '__main__':
#     DATASET_PATH = './data/code_blocks_regex_graph_v2.1.csv'
#     MODEL_DIR = './models/svm_regex_{}.sav'.format('graph_v2.1')
#     TFIDF_DIR = './models/tfidf_svm_graph_v2.1.pickle'
#     CODE_COLUMN = 'code_block'
#     TAG_TO_PREDICT = 'preprocessing'
#     SCRIPT_DIR = __file__
    
#     df, code_blocks = load_code_blocks(DATASET_PATH, CODE_COLUMN)
#     nrows = df.shape[0]
#     print("loaded")
#     tfidf_params = {'min_df': 5
#             , 'max_df': 0.3
#             , 'smooth_idf': True}
#     SVM_params = {'C':100
#             , 'kernel':"linear"
#             , 'random_state':241}
#     data_meta = {'DATASET_PATH': DATASET_PATH
#                 ,'nrows': nrows
#                 ,'label': TAG_TO_PREDICT
#                 ,'model': MODEL_DIR
#                 ,'source': SCRIPT_DIR}

#     with dagshub.dagshub_logger() as logger:
#         print("evaluating..")
#         metrics = SVM_evaluate(df, code_blocks, tfidf_params, TFIDF_DIR, SVM_params)
#         print("saving the results..")
#         logger.log_hyperparams(data_meta)
#         logger.log_hyperparams(tfidf_params)
#         logger.log_hyperparams(SVM_params)
#         logger.log_metrics(metrics)
#     print("finished")

In [23]:
if __name__ == '__main__':
    GRAPH_VER = 6
    SCRIPT_DIR = 'svm_classifier.ipynb'
    CODE_COLUMN = 'code_block'
    TAG_TO_PREDICT = 'preprocessing'
    
    DATASET_PATH = './data_mimi/code_blocks_regex_graph_v{}.csv'.format(GRAPH_VER)
    MODEL_DIR = './models/svm_regex_graph_v{}.sav'.format(GRAPH_VER)
    TFIDF_DIR = './models/tfidf_svm_graph_v{}.pickle'.format(GRAPH_VER)
    clf = pickle.load(open(MODEL_DIR, 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: './models/svm_regex_graph_v6.sav'

In [17]:
tfidf = pickle.load(open(TFIDF_DIR, 'rb'))
len(tfidf.vocabulary_)

48993

In [18]:
clf = clf.estimators_[0]
print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =    (0, 42663)	0.2821803513963908
  (0, 38897)	0.0578793552624784
  (0, 34834)	0.42017347249558573
  (0, 9311)	0.21245733117566468
  (0, 43940)	0.15338406143525404
  (0, 29173)	0.13013941337929638
  (0, 27308)	0.1415896144461506
  (0, 36656)	0.40328679169596965
  (0, 24597)	0.19471104664691843
  (0, 20430)	0.2041941597770926
  (0, 43470)	0.16676999931828385
  (0, 25289)	0.026682728182567384
  (0, 24306)	0.17047763978914707
  (0, 24302)	0.07512988844509241
  (0, 20761)	0.027522619398377215
  (0, 19099)	0.18557357271201963
  (0, 12118)	0.10673091273026954
  (0, 11934)	0.057595451905092555
  (0, 3386)	0.19889057975400493
  (0, 13901)	0.16747753402156684
  (0, 13900)	0.14411357721598567
  (0, 12602)	0.23599929970054376
  (0, 21965)	0.0770342580117467
  (0, 21864)	0.07770596159892064
  (0, 14984)	0.08130603248965262
  :	:
  (0, 44215)	0.4471575551619707
  (0, 42412)	0.12442903072489733
  (0, 42030)	0.18308885987641638
  (0, 38731)	0.12769133362449783
  (0, 38411)	0.052486464817444525
  (

In [19]:
w = clf.coef_
weights = w.toarray()[0]

In [20]:
vocab = list(tfidf.vocabulary_.keys())
vocab_freq = list(tfidf.vocabulary_.values())

In [21]:
interpret = pd.DataFrame()
interpret['vocab'] = vocab
interpret['vocab_freq'] = vocab_freq
interpret['weights'] = weights

In [22]:
interpret.to_csv('./model_interpretation/svm_interpret_v{}.csv'.format(GRAPH_VER), index=False)

FileNotFoundError: [Errno 2] No such file or directory: './model_interpretation/svm_interpret_v2.csv'

In [12]:
interpret.sort_values(by='weights', ascending=False)[interpret['vocab_freq'] > 48000].head(10)

,vocab,vocab_freq,weights
42649,y_val1,48302,5.496884
37343,y_testl11,48174,3.836949
39589,y_test_cc_lgb,48135,3.516085
21676,y_values,48331,1.580852
41962,y_trfm,48278,1.015931
12312,y_pred_in,48013,0.956726
27964,y_pred_prob_yes,48026,0.940454
42648,y_train4,48189,0.895213
282,zeros,48648,0.882440
21766,ys_train,48565,0.777850


In [13]:
interpret.sort_values(by='vocab_freq', ascending=False)[interpret['weights'] > 3]

,vocab,vocab_freq,weights
42649,y_val1,48302,5.496884
37343,y_testl11,48174,3.836949
39589,y_test_cc_lgb,48135,3.516085
9856,traumatized,44062,3.271442
9434,trainset,43933,3.736686
43744,switzerland_nan,40674,9.559917
9848,storm,39953,9.684152
41675,personal_yards,31962,4.383437
35928,nan_new,28789,4.212826
33641,modelu2os,27808,8.978572
